In [1]:
from jupyter_init import setup

setup()

from src_code.config import *

## Loading Dataset

In [2]:
import pandas as pd
import numpy as np

TRANSFORMED_DF = DEFECTORS_DIR / "jit_bug_prediction_splits" / "time" / "train_labeled_features_partial.feather"

# ---- LOAD ----
df = pd.read_feather(TRANSFORMED_DF)
print(f"Loaded dataframe with {len(df)} rows and {len(df.columns)} columns\n")

df.dtypes

Loaded dataframe with 16017 rows and 31 columns



datetime                      datetime64[us, pytz.FixedOffset(-120)]
commit                                                        object
repo                                                          object
filepath                                                      object
content                                                       object
methods                                                       object
lines                                                         object
author_email                                                  object
canonical_datetime                               datetime64[ns, UTC]
author_exp_pre                                                 int64
author_recent_activity_pre                                     int64
label                                                          int64
loc_added                                                      int64
loc_deleted                                                    int64
files_changed                     

## Converting NumpyArray -> List



In [3]:
#Convert the NumPy arrays back to Python lists
for col in ['code_embed', 'msg_embed', 'methods', 'lines']:
    # Use .apply(list) or .apply(lambda x: x.tolist()) for robustness
    df[col] = df[col].apply(list)

# print(df['content'].head(5))

## Missing Value Audit

In [4]:
print("## 1. Missing Values per Column")
nulls = df.isnull().sum().sort_values(ascending=False)
print(nulls.to_markdown())

## 1. Missing Values per Column
|                            |   0 |
|:---------------------------|----:|
| datetime                   |   0 |
| commit                     |   0 |
| repo                       |   0 |
| filepath                   |   0 |
| content                    |   0 |
| methods                    |   0 |
| lines                      |   0 |
| author_email               |   0 |
| canonical_datetime         |   0 |
| author_exp_pre             |   0 |
| author_recent_activity_pre |   0 |
| label                      |   0 |
| loc_added                  |   0 |
| loc_deleted                |   0 |
| files_changed              |   0 |
| hunks_count                |   0 |
| msg_len                    |   0 |
| has_fix_kw                 |   0 |
| has_bug_kw                 |   0 |
| ast_delta                  |   0 |
| complexity_delta           |   0 |
| max_func_change            |   0 |
| time_since_last_change     |   0 |
| todo                       |   0 |
| fixm

## Primary Key Integrity

In [5]:
print("## 2. Primary Key Uniqueness Check")
key_cols = ["repo", "commit", "filepath"]

dupes = df.duplicated(subset=key_cols).sum()
print(f"Duplicate key rows: {dupes}")

## 2. Primary Key Uniqueness Check
Duplicate key rows: 0


## Label Distribution

In [6]:
print("## 3. Label Distribution")
print(df['label'].value_counts(normalize=True).to_markdown())

## 3. Label Distribution
|   label |   proportion |
|--------:|-------------:|
|       0 |      0.51052 |
|       1 |      0.48948 |


## Repository Distribution (Imbalance Check)

In [7]:
print("## 4. Repository Distribution")
repo_dist = df['repo'].value_counts(normalize=True)
print(repo_dist.to_markdown())

## 4. Repository Distribution
| repo    |   proportion |
|:--------|-------------:|
| pandas  |    0.90635   |
| airflow |    0.0936505 |


## Value Range Scan for Numeric Columns

Automatically detects:

- negatives where not allowed
- max values
- suspicious spikes

In [8]:
print("## 5. Numeric Column Range Scan")
num_cols = df.select_dtypes(include=[np.number]).columns

ranges = pd.DataFrame({
    "min": df[num_cols].min(),
    "median": df[num_cols].median(),
    "mean": df[num_cols].mean(),
    "max": df[num_cols].max()
})

print(ranges.to_markdown())

## 5. Numeric Column Range Scan
|                            |     min |   median |          mean |              max |
|:---------------------------|--------:|---------:|--------------:|-----------------:|
| author_exp_pre             |       0 |      113 |   529.822     |   3557           |
| author_recent_activity_pre |       0 |       15 |    37.8364    |    238           |
| label                      |       0 |        0 |     0.48948   |      1           |
| loc_added                  |       0 |       33 |    76.4645    |  11824           |
| loc_deleted                |       0 |       57 |   115.242     |  12067           |
| files_changed              |       0 |        6 |    13.0631    |    162           |
| hunks_count                |       0 |       28 |    64.1849    |   1104           |
| msg_len                    |       7 |       60 |   112.901     |   2858           |
| has_fix_kw                 |       0 |        0 |     0.183243  |      1           |
| has_bug_k

## Check Columns Expected to Be Non-Negative

In [9]:
non_negative_cols = [
    "loc_added", "loc_deleted",
    "files_changed", "hunks_count",
    "msg_len", "ast_delta",
    "complexity_delta", "max_func_change",
    "author_exp_pre", "author_recent_activity_pre",
    "todo", "fixme", "try", "except", "raise",
    "recent_churn"
]

print("## 6. Negative Value Check")
for col in non_negative_cols:
    bad = (df[col] < 0).sum()
    print(f"{col}: {bad} negative values")

## 6. Negative Value Check
loc_added: 0 negative values
loc_deleted: 0 negative values
files_changed: 0 negative values
hunks_count: 0 negative values
msg_len: 0 negative values
ast_delta: 0 negative values
complexity_delta: 0 negative values
max_func_change: 0 negative values
author_exp_pre: 0 negative values
author_recent_activity_pre: 0 negative values
todo: 0 negative values
fixme: 0 negative values
try: 0 negative values
except: 0 negative values
raise: 0 negative values
recent_churn: 0 negative values


## Suspicious Feature Check: time_since_last_change

In [10]:
print("## 7. time_since_last_change Outliers")
tslc = df["time_since_last_change"]

print(f"Negative values: {(tslc < 0).sum()}")
print(f"99.9% quantile: {tslc.quantile(0.999)}")
print(f"Min: {tslc.min()}")
print(f"Max: {tslc.max()}")

## 7. time_since_last_change Outliers
Negative values: 3
99.9% quantile: 1403131.112000024
Min: -396818
Max: 20466389


### Understanding the Feature

*time_since_last_change = c.committed_date - last_time*

Where:
- c.committed_date = current commit timestamp (UNIX seconds)
- last_time = timestamp of first parent commit

So the feature = time difference between consecutive commits.

This represents how much time passed between commits in a repo.

### Why Negative Values?

Meaning:

- Some commits appear to be ~4.6 days negative (-396,818 sec)
- Some commits appear to be ~77 days ahead (6.6M sec)

This is expected when real Git data is used.

Git timestamps can go backwards because:

#### (1) Rebased or rewritten history

During rebases, old commits appear “later” than newer ones.

*WHY?*

When you merge/rebase, Git does not reorder commits chronologically.
Instead, it preserves the logical order of development.

#### (2) Merge parents

You use only the first parent:

if c.parents:
    last_time = c.parents[0].committed_date


But merges may introduce non-linear time ordering.

#### (3) Clock drift

Different authors → different local machine clocks.

#### (4) Shallow clones or incomplete history

If the repo is shallow-fetched, parent commits may have weird timestamps.

None of this indicates your extraction is wrong.



## Binary Flag Integity

In [11]:
print("## 8. Binary Columns Integrity")
bin_cols = ["has_fix_kw", "has_bug_kw"]

for col in bin_cols:
    bad = df[~df[col].isin([0,1])]
    print(f"{col}: {len(bad)} invalid values")

## 8. Binary Columns Integrity
has_fix_kw: 0 invalid values
has_bug_kw: 0 invalid values


## Embedding Consistency Check

Ensure:

- no None
- all lists
- identical dimensionality

In [12]:
# # Convert the NumPy arrays back to Python lists
# for col in ['code_embed', 'msg_embed']:
#     # Use .apply(list) or .apply(lambda x: x.tolist()) for robustness
#     df[col] = df[col].apply(list)

print(type(df.loc[0, 'code_embed']))
print(type(df.loc[0, 'msg_embed']))

print("## 9. Embedding Structural Checks")

# None count
print("code_embed None count:", df['code_embed'].isna().sum())
print("msg_embed None count:", df['msg_embed'].isna().sum())

# Check if all are lists
print("\nNon-list code_embed rows:", (~df['code_embed'].apply(lambda x: isinstance(x, list))).sum())
print("Non-list msg_embed rows:", (~df['msg_embed'].apply(lambda x: isinstance(x, list))).sum())

# Check dimensionality
dims = df['code_embed'].apply(lambda x: len(x) if isinstance(x, list) else None)
print("\nEmbedding dimensionality distribution:")
print(dims.value_counts().head())

<class 'list'>
<class 'list'>
## 9. Embedding Structural Checks
code_embed None count: 0
msg_embed None count: 0

Non-list code_embed rows: 0
Non-list msg_embed rows: 0

Embedding dimensionality distribution:
code_embed
768    16017
Name: count, dtype: int64


## Datetime Consistency

Check for:

- NaT values
- ordering sanity (commit should not be older than file's previous record)

In [13]:
print("## 10. Datetime Columns Audit")

date_cols = ["datetime", "canonical_datetime"]

for col in date_cols:
    print(f"{col}: NaT count = {df[col].isna().sum()}")
    print(f"{col}: min = {df[col].min()}, max = {df[col].max()}")

## 10. Datetime Columns Audit
datetime: NaT count = 0
datetime: min = 2009-11-21 20:57:45-02:00, max = 2022-01-03 09:36:56-02:00
canonical_datetime: NaT count = 0
canonical_datetime: min = 2009-11-21 22:57:45+00:00, max = 2022-01-03 21:36:56+00:00


## Correlation Hot Spots
To detect redundant or suspiciously identical columns.

In [29]:
# print("## 11. Correlations (Numeric Only)")
# corr = df[num_cols].corr()
# # display(corr.style.background_gradient(cmap='coolwarm'))
# matrix = corr.style.background_gradient(cmap='coolwarm')
# matrix

print("## 11. Correlations (Numeric Only)")
corr = df[num_cols].corr()
print(corr.to_markdown())  # nicely formatted markdown table

## 11. Correlations (Numeric Only)
|                            |   author_exp_pre |   author_recent_activity_pre |        label |   loc_added |   loc_deleted |   files_changed |   hunks_count |     msg_len |   has_fix_kw |   has_bug_kw |   ast_delta |   complexity_delta |   max_func_change |   time_since_last_change |        todo |       fixme |          try |      except |       raise |   recent_churn |
|:---------------------------|-----------------:|-----------------------------:|-------------:|------------:|--------------:|----------------:|--------------:|------------:|-------------:|-------------:|------------:|-------------------:|------------------:|-------------------------:|------------:|------------:|-------------:|------------:|------------:|---------------:|
| author_exp_pre             |        1         |                   0.723431   |  0.110553    |  -0.062186  |   -0.103806   |     -0.0929121  |   -0.0922295  | -0.143356   |  -0.160424   |  -0.0467965  |  -0.0256155 |

In [15]:
print("## 12. Columns With Only One Unique Value")

for col in df.columns:
    # Convert arrays to tuples for uniqueness check
    if df[col].apply(lambda x: isinstance(x, (np.ndarray, list))).any():
        # Convert each element to a tuple (or a string representation)
        # print(col)
        unique_values = df[col].apply(lambda x: tuple(x) if isinstance(x, (np.ndarray, list)) else x)
    else:
        unique_values = df[col]

    if unique_values.nunique(dropna=True) == 1:
        print(f"⚠️ {col} has only one unique value")

## 12. Columns With Only One Unique Value


## Check Text Columns for Weirdness

Empty strings? Too short? Too long?

In [16]:
print("## 13. Text Field Checks")

if 'content' in df.columns:
    print("Empty content rows:", (df['content'].str.len() == 0).sum())
    print(df['content'].str.len().describe())

# Check 'methods' if it is a list column
if 'methods' in df.columns:
    # Use len() on the Python lists
    print("\nEmpty methods rows:", (df['methods'].apply(len) == 0).sum())
    print(df['methods'].apply(len).describe())

## 13. Text Field Checks
Empty content rows: 41
count     16017.000000
mean       1770.271337
std        3000.898475
min           0.000000
25%         472.000000
50%         890.000000
75%        1889.000000
max      109324.000000
Name: content, dtype: float64

Empty methods rows: 2722
count    16017.000000
mean         2.620528
std          4.916241
min          0.000000
25%          1.000000
50%          1.000000
75%          3.000000
max        148.000000
Name: methods, dtype: float64


## Check For Impossible Values

In [17]:
print("## 14. Logical Consistency Checks")

# msg_len should match commit message length
if "msg_len" in df.columns:
    print("msg_len outliers (msg_len <= 0):", (df['msg_len'] <= 0).sum())

## 14. Logical Consistency Checks
msg_len outliers (msg_len <= 0): 0


## Filepath Sanity
Check for Windows vs POSIX weirdness.

In [22]:
# Normalize all filepaths to use forward slashes
df['filepath'] = df['filepath'].str.replace('\\', '/', regex=False)

# Check again for paths without a slash
bad_paths = df[~df['filepath'].str.contains('/')]
print("Filepaths without / (unexpected after normalization):", len(bad_paths))
print(df.loc[~df['filepath'].str.contains('/'), 'filepath'].head(20))

Filepaths without / (unexpected after normalization): 237
11        setup.py
80        setup.py
81     setupegg.py
122       setup.py
123       setup.py
138       setup.py
144       setup.py
168       setup.py
278       setup.py
461       setup.py
462       setup.py
479       setup.py
497       setup.py
513       setup.py
514       setup.py
523       setup.py
524       setup.py
531       setup.py
562       setup.py
570       setup.py
Name: filepath, dtype: object


## Check Recent Churn for Extreme Outliers

In [23]:
print("## 16. recent_churn Outlier Scan")
print(df['recent_churn'].describe())
print("99.9% quantile:", df['recent_churn'].quantile(0.999))

## 16. recent_churn Outlier Scan
count     16017.000000
mean       6416.870388
std       10498.605904
min           0.000000
25%         234.000000
50%        2072.000000
75%        9027.000000
max      110348.000000
Name: recent_churn, dtype: float64
99.9% quantile: 93818.97600000042


## Check Distribution of Code Activity Keywords

(todo, fixme, try/except/raise)

In [24]:
print("## 17. Keyword Column Distributions")
kw_cols = ["todo", "fixme", "try", "except", "raise"]

print(df[kw_cols].describe().T.to_markdown())

## 17. Keyword Column Distributions
|        |   count |      mean |      std |   min |   25% |   50% |   75% |   max |
|:-------|--------:|----------:|---------:|------:|------:|------:|------:|------:|
| todo   |   16017 | 0.166635  | 0.741512 |     0 |     0 |     0 |     0 |    26 |
| fixme  |   16017 | 0.0176687 | 0.267961 |     0 |     0 |     0 |     0 |    17 |
| try    |   16017 | 0.519698  | 2.60688  |     0 |     0 |     0 |     0 |    56 |
| except |   16017 | 1.12961   | 6.96136  |     0 |     0 |     0 |     0 |   102 |
| raise  |   16017 | 1.77436   | 7.42676  |     0 |     0 |     0 |     1 |   213 |
